In [ ]:
!wget https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel

--2024-06-26 05:42:26--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55156 (54K)
Saving to: ‘integrated_call_samples_v3.20130502.ALL.panel.1’

integrated_call_sam 100%[===================>]  53.86K   279KB/s    in 0.2s    

2024-06-26 05:42:27 (279 KB/s) - ‘integrated_call_samples_v3.20130502.ALL.panel.1’ saved [55156/55156]



In [ ]:
!wget -nH --cut-dirs=4 -r -p -e robots=off -A 'ALL.*' https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/

--2024-06-26 05:39:20--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/
Resolving ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)... 193.62.193.167
Connecting to ftp.1000genomes.ebi.ac.uk (ftp.1000genomes.ebi.ac.uk)|193.62.193.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html.tmp’

index.html.tmp          [ <=>                ]  19.90K  --.-KB/s    in 0.1s    

2024-06-26 05:39:21 (208 KB/s) - ‘index.html.tmp’ saved [20382]

Removing index.html.tmp since it should be rejected.

--2024-06-26 05:39:21--  https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/?C=N;O=D
Reusing existing connection to ftp.1000genomes.ebi.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html?C=N;O=D.tmp’

index.html?C=N;O=D.     [ <=>                ]  19.90K  --.-KB/s    in 0.03s   

2024-06-26 05:39:21 (642 KB/s) - ‘index.html?C=N;O=D.tmp’ saved [2038

Практическая работа. Работа с библиотекой.

In [ ]:
pip install hail

In [ ]:
import hail as hl
from hail.plot import show
from pprint import pprint

Prepare - удаляем группу AMR (смешанные американцы) и создаем тестовую и обучающую выборки

In [ ]:
!grep -E 'AFR|EUR|SAS|EAS' annotation.txt > annotation_noAMR.txt

grep: annotation.txt: No such file or directory


STEP0 - отбираем только би-аллельные SNP и убирает инделы

In [ ]:
chr1 = hl.import_vcf('ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz', force_bgz=True)

Initializing Hail with default parameters...
Running on Apache Spark version 3.5.1
SparkUI available at http://d41f68cc0b17:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.131-11d9b2ff89da
LOGGING: writing to /content/hail-20240626-0543-0.2.131-11d9b2ff89da.log


In [ ]:
#only bi-allelic SNP
chr1 = chr1.filter_rows(hl.len(chr1.alleles) == 2)
#remove indels
chr1 = chr1.filter_rows(hl.is_snp(chr1.alleles[0], chr1.alleles[1]))

In [ ]:
#Получаем тестовую и обучающую выборку
import pandas as pd
import numpy as np
import os
#Обозначение переменных
testindsperdeme = 100
demes = 5
filenameInds = "integrated_call_samples_v3.20130502.ALL.panel"
filenameIndsTest = "annotationTest.txt"
filenameIndsTraining = "annotationTraining.txt"


size = [testindsperdeme for i in range(demes)]
a = pd.read_csv(filenameInds, sep='\t' )
pops = np.sort(np.unique(a.iloc[:, 2]))
inds = a.iloc[:, [0, 2]]
noinds = np.zeros(len(pops), dtype=int)
testset, trainingset = [], []

for i in range(len(pops)):
    indslocal = inds[inds.iloc[:, 1] == pops[i]].iloc[:, 0]
    noinds[i] = np.sum(inds.iloc[:, 1] == pops[i])
    s = np.random.choice(noinds[i], size[i], replace=False)
    testset.extend(indslocal.iloc[s])
    trainingset.extend(indslocal.iloc[np.setdiff1d(np.arange(noinds[i]), s)])

#Создаем пустые файлы с заголовком
os.system(f" head -n 1 {filenameInds} > {filenameIndsTest}")
os.system(f" head -n 1 {filenameInds} > {filenameIndsTraining}")

#Создаем файлы с аннотацией образцов для обучающего и тестового образца по отобранным именам
for test_id in testset:
    os.system(f"grep -w {test_id} {filenameInds} >> {filenameIndsTest}")

for train_id in trainingset:
    os.system(f"grep -w {train_id} {filenameInds} >> {filenameIndsTraining}")

STEP1 - считаем частоты SNP

In [ ]:
table = hl.import_table("integrated_call_samples_v3.20130502.ALL.panel").key_by("sample")
mt = chr1

# объединяем данные из аннотации с данными из vcf файла
mt = mt.annotate_cols(pheno=table[mt.s])

#Список популяций в файле
pops = list(table.aggregate(hl.agg.collect_as_set(table.super_pop)))

# Инициализация таблицы первой популяцией
mt_pop = mt.filter_cols(mt.pheno.super_pop == pops[0])
mt_pop = hl.variant_qc(mt_pop)
mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AF[1])
freq_table = mt_pop.rows().select('alt').rename({"alt": f'{pops[0]}'})

# Цикл по оставшимся популяциям
for pop in pops[1:]:
  mt_pop = mt.filter_cols(mt.pheno.super_pop == pop)
  mt_pop = hl.variant_qc(mt_pop)
  mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AF[1])
  pop_freq = mt_pop.rows().select('alt').rename({"alt": f'{pop}'})

  # Объединяем таблицы
  freq_table = freq_table.join(pop_freq)

STEP2 - фильтрация SNP по информативности

In [ ]:
#Задаем уровень информативности, при котором мы оставляем SNP
informativenessBound = 0.9
#Вводим дополнительные функции для подсчета информативности
def xlogx2(x):
    z = xlogx(x) + xlogx(1 - x)
    return z

def xlogx(x):
    z = hl.if_else(x == 0, 0, x * hl.log(x))
    return z
#Считаем количесвто строк в файле
nss = freq_table.count()
#Считаем средние
freq_table = freq_table.annotate(
    mean_row=(freq_table.AFR + freq_table.SAS + freq_table.EUR + freq_table.AMR + freq_table.EAS) / len(pops)
)
freq_table = freq_table.annotate(
    mean_row_log=(xlogx2(freq_table.AFR) + xlogx2(freq_table.SAS) + xlogx2(freq_table.EUR) + xlogx2(freq_table.AMR) + xlogx2(freq_table.EAS)) / len(pops)
)
#Считаем информативность
freq_table = freq_table.annotate(info= -xlogx2(freq_table.mean_row) + freq_table.mean_row_log)
#Фильтруем таблицу по информативности (в порядке убывания) и отбираем 10% самых информативных SNP
informativenessQuantile = freq_table.order_by(hl.desc(freq_table.info)).head(hl.eval(hl.int32(hl.floor((1 - informativenessBound)* nss))))
#Создаем новый vcf-файл с отобранными SNP по информативности
informativenessQuantile = informativenessQuantile.key_by("locus")
mt = chr1.filter_rows(hl.is_defined(informativenessQuantile[chr1.locus]))
#Сохраняем файл
hl.export_vcf(mt, 'tmp/filter_chr1.vcf.bgz')

STEP3 - делим имеющийся файл на маленькие

In [ ]:
mt = hl.import_vcf('tmp/filter_chr1.vcf.bgz')
stepsize = 10000
# Вычисляем количество сегментов и преобразуем в список
segments = hl.range(0, hl.int32((mt.count_rows() - 1) // stepsize + 1)).collect()
# Добавляем столбец с номером строки
mt = mt.annotate_rows(row_index=hl.scan.count())

# Фильтруем строки по номеру

# Цикл по сегментам
for j in segments[0]:
    start = j * stepsize
    end = hl.eval(min((j + 1) * stepsize, mt.count_rows()))
    # Фильтруем строки по номеру
    segment_mt = mt.filter_rows((mt.row.row_index >= start) & (mt.row.row_index < end))
    hl.export_vcf(segment_mt, f"tmp/chr_segment{j}.vcf.bgz")

STEP4 - считаем кол-во строк в новом vcf файле

In [ ]:
nss = hl.eval(chr.count()[0])

In [ ]:
#STEP5
def genotype_to_number(gt):
    z = hl.case().when(gt.is_het(), 1).when(gt.is_hom_ref(), 0).when(gt.is_hom_var(), 2).or_missing()
    return z
for j in segments[0]:
    mt = hl.import_vcf(f"tmp/chr_segment{j}.vcf.bgz")
    mt = mt.annotate_entries(GT=genotype_to_number(mt.GT))
    mt.write(f'tmp/chr_segment{j}.mt', overwrite=True)

In [ ]:
chr = chr1.head(200)

In [ ]:
def genotype_to_number(gt):
    z = hl.case().when(gt.is_het(), 1).when(gt.is_hom_ref(), 0).when(gt.is_hom_var(), 2).or_missing()
    return z

In [ ]:
mt = chr.annotate_entries(GT=genotype_to_number(chr.GT))
mt.write('chr.mt', overwrite=True)

2024-06-26 05:44:34.832 Hail: INFO: scanning VCF for sortedness...
2024-06-26 05:47:21.919 Hail: INFO: Coerced prefix-sorted VCF, requiring additional sorting within data partitions on each query.
2024-06-26 05:47:30.830 Hail: INFO: wrote matrix table with 200 rows and 2504 columns in 1 partition to chr.mt


In [ ]:
#STEP6
a = hl.import_table("annotationTraining.txt").select("sample","super_pop")
#Создаем список с именами все анализируемых vcf файлов
#findAIMs
method = "naiveBayes"
error_type="misclassification"
diploid = True
outfile = "AIMs"
#Количесвто AIM, которое мы хотим получить в итоге
numAIM = 10
#Частота встречаемости популяций
samples_in_deme = a.aggregate(hl.agg.counter(a.super_pop))
#Создаем пустую таблицу для хоанения информативных маркеров
sampleAIMs = hl.Table.parallelize([])
#Считаем количесвто столбцов в таблице sampleAIMs
no = hl.if_else(len(sampleAIMs.row) == 0, 0, len(sampleAIMs.row))
min_error = 1000

2024-06-26 05:47:42.930 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']
2024-06-26 05:47:42.941 Hail: INFO: Reading table without type imputation
  Loading field 'sample' as type str (not specified)
  Loading field 'pop' as type str (not specified)
  Loading field 'super_pop' as type str (not specified)
  Loading field 'gender' as type str (not specified)


In [ ]:
#Внутреннее содержимое функции getfreqs
#Создаем список всех локусов
locus_list = mt.locus.collect()
#Импортируем необходимые файлы
table_annotation = hl.import_table("annotationTraining.txt").key_by("sample")
mt = chr

#объединяем данные из аннотации с данными из vcf файла
mt = mt.annotate_cols(pheno=table_annotation[mt.s])

#Список популяций в файле
pops = list(table_annotation.aggregate(hl.agg.collect_as_set(table_annotation.super_pop)))

# Инициализация таблицы первой популяцией
mt_pop = mt.filter_cols(mt.pheno.super_pop == pops[0])
mt_pop = hl.variant_qc(mt_pop)
mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AC[1]).key_rows_by("locus")
freq_list = mt_pop.alt.collect()
locus_freq_table = hl.Table.parallelize(
    [{"locus": locus, "frequency": freq} for locus, freq in zip(locus_list, freq_list)],
    hl.tstruct(locus=hl.tlocus("GRCh37"), frequency=hl.tint32),
).rename({"frequency": f'{pops[0]}'})
table = locus_freq_table.key_by("locus")
# Цикл по оставшимся популяциям
for pop in pops[1:]:
  mt_pop = mt.filter_cols(mt.pheno.super_pop == pop)
  mt_pop = hl.variant_qc(mt_pop)
  mt_pop = mt_pop.annotate_rows(alt = mt_pop.variant_qc.AC[1]).key_rows_by("locus")
  freq_list = mt_pop.alt.collect()
  locus_freq_table = hl.Table.parallelize(
    [{"locus": locus, "frequency": freq} for locus, freq in zip(locus_list, freq_list)],
    hl.tstruct(locus=hl.tlocus("GRCh37"), frequency=hl.tint32)).rename({"frequency": f'{pop}'}).key_by("locus")
  table = table.join(locus_freq_table, how="left")

2024-06-26 05:48:51.475 Hail: INFO: Reading table without type imputation
  Loading field 'sample' as type str (not specified)
  Loading field 'pop' as type str (not specified)
  Loading field 'super_pop' as type str (not specified)
  Loading field 'gender' as type str (not specified)


In [ ]:
#Считаем количесвто групп
demes = len(table.row) - 1

In [ ]:
#Реализуем функцию find.next.AIM

In [ ]:
import re

In [ ]:
sample_test = hl.read_matrix_table('chr.mt').head(20)
sample_names = [(re.search(r"Struct\(sample='(.*?)'\)", str(i)).group(1)) for i in table_annotation.key_by().select("sample").collect()]

In [ ]:
sample_test.count_rows()

20

In [ ]:
#нормальный log.prediction naive bayes
def log_prediction_naiveBayes(sample_test, table_with_freq, sample_names, number_of_snp):
  res = np.zeros((len(sample_names), 5))
  for j in range(len(sample_names)):
    selected_mt = sample_test.filter_cols(sample_test.s == f'{sample_names[j]}').GT.collect()[number_of_snp]
    for k in range(5):
      counts = [int(re.search(r'\d+(\.\d+)?', str(i)).group(0)) for i in table_with_freq.select(f"{pops[k]}").collect()][number_of_snp]
      res[j, :][k] += np.log(1 + counts) * selected_mt + np.log((1 + diploid) * samples_in_deme[pops[k]] - counts + 1) * (1 + diploid - selected_mt)
  else:
    return res

In [ ]:
def prediction_naive_bayes(sample_test, counts, sample_names, number_of_snp):
  res = log_prediction_naiveBayes(sample_test, counts, sample_names, number_of_snp)
  # Вычисляем экспоненту для каждого значения в res, вычитая максимальное значение
  res = np.exp(res - np.max(res))
  # Нормализуем каждую строку так, чтобы сумма элементов в строке была равна 1
  res = res / res.sum(axis=1, keepdims=True)
  return res

In [ ]:
def logloss_error(prediction_table):
  """
  Вычисляет logloss error для Hail Table с вероятностями.

  Args:
      prediction_table (Hail Table): Таблица, содержащая столбцы с вероятностями
          принадлежности к популяциям и столбец 'super_pop' с истинными классами.

  Returns:
      float: Значение logloss error.
  """

  # Получаем список названий популяций из колонок
  populations = ['EAS', 'SAS', 'AFR', 'EUR', 'AMR']

  # Создаем список выражений для получения вероятности для истинного класса
  probs_for_correct_classes_expr = [
    hl.if_else(prediction_table.super_pop == population,
               prediction_table[population],
               hl.literal(0))
    for population in populations
  ]

  # Вычисляем среднее логарифма вероятности для истинного класса
  logloss = - prediction_table.aggregate(hl.agg.mean(hl.log(hl.sum(probs_for_correct_classes_expr))))

  return logloss

In [ ]:
def App_test_error(sample_test, counts, sample_names, number_of_snp, table):
    """
  Вычисляет ошибку для каждого SNP

  Args:
     sample_test - matrix table по типу vcf.gz только генотипы 0, 1, 2 - получен на 5м шаге
     counts - hail table с частотами каждого SNP в популяциях
     sample_names - имя образцов для обучающей выборки
     number_of_snp - для какого SNP считаем вероятности
     table - hail table = annotationTraining.txt

  Returns:
    Значение logloss error для каждого SNP в файле.
  """
    #Считаем матрицу вероятностей
    res = prediction_naive_bayes(sample_test, counts, sample_names, number_of_snp)
    # Преобразуем numpy массив в pandas DataFrame
    df = pd.DataFrame(res, columns=pops)
    #Добавляем колонку с именами образцов
    df["sample"] = sample_names
    #Добавляем колонку с именами популяций
    population = table.super_pop.collect()
    df["super_pop"] = population
    ht = hl.Table.from_pandas(df)
    error = logloss_error(ht)
    return error

In [ ]:
#продолжение find next AIMs - мы получаем список error для каждого SNP
min_error = min(error)
best_snp = error.index(min_error) #находим образец с минимальной ошибкой
sample_next_AIM = sample_test.filter_cols(sample_test.s == f'{sample_names[best_snp]}').GT
#return sample_next_AIM, min_error

In [ ]:
#Собираем функцию find AIM
for i in range(numAIM):
  best, error = find_next_AIM()


Отбросы кода

In [ ]:
#log.prediction.naive.bayes
t = hl.import_table("annotationTraining.txt")
sample_test = hl.read_matrix_table('chr.mt')
table_with_freq = table.key_by()
sample_names = [(re.search(r"Struct\(sample='(.*?)'\)", str(i)).group(1)) for i in t.select("sample").collect()]
res = np.zeros((len(sample_names), demes))
for j in range(len(sample_names)):
  selected_mt = sample_test.filter_cols(sample_test.s == f'{sample_names[j]}').GT.collect()
  for k in range(5):
    counts = [int(re.search(r'\d+(\.\d+)?', str(i)).group(0)) for i in table_with_freq.select(f"{pops[k]}").collect()]
    for i in range(1):
      res[j, :][k] += np.log(1 + counts[i]) * selected_mt[i] + np.log((1 + diploid) * samples_in_deme[pops[k]] - counts[i] + 1) * (1 + diploid - selected_mt[i])

2024-06-25 13:19:14.878 Hail: INFO: Reading table without type imputation
  Loading field 'sample' as type str (not specified)
  Loading field 'pop' as type str (not specified)
  Loading field 'super_pop' as type str (not specified)
  Loading field 'gender' as type str (not specified)
2024-06-25 13:19:16.530 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:17.088 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:17.599 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:18.088 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:18.662 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:20.652 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:20.919 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:21.185 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:21.444 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:21.709 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:23.005 Hail: INFO: Coerced sorted dataset
2024-06-25 13:19:23.247 Hail: INFO: Coerced sorted dataset
2024-0

KeyboardInterrupt: 

In [ ]:
#нормальный log.prediction naive bayes
t = hl.import_table("annotationTraining.txt")
sample_test = hl.read_matrix_table('chr.mt')
table_with_freq = table.key_by()
sample_names = [(re.search(r"Struct\(sample='(.*?)'\)", str(i)).group(1)) for i in t.select("sample").collect()]
res = np.zeros((len(sample_names), demes))
#Отдельно вводим число, до этой функции
number_of_snp = 3
for j in range(len(sample_names)):
  selected_mt = sample_test.filter_cols(sample_test.s == f'{sample_names[j]}').GT.collect()[number_of_snp]
  for k in range(5):
    counts = [int(re.search(r'\d+(\.\d+)?', str(i)).group(0)) for i in table_with_freq.select(f"{pops[k]}").collect()][number_of_snp]
    res[j, :][k] += np.log(1 + counts) * selected_mt + np.log((1 + diploid) * samples_in_deme[pops[k]] - counts + 1) * (1 + diploid - selected_mt)

2024-06-25 13:26:36.059 Hail: INFO: Reading table without type imputation
  Loading field 'sample' as type str (not specified)
  Loading field 'pop' as type str (not specified)
  Loading field 'super_pop' as type str (not specified)
  Loading field 'gender' as type str (not specified)
2024-06-25 13:26:38.358 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:38.916 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:39.391 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:39.726 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:40.026 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:41.180 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:41.880 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:42.542 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:43.208 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:43.974 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:46.601 Hail: INFO: Coerced sorted dataset
2024-06-25 13:26:47.051 Hail: INFO: Coerced sorted dataset
2024-0

KeyboardInterrupt: 